# Proceso ETL Vuelos_DB


## 1. Importar paquetes requeridos

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf, isnan, when, count, col, to_date, regexp_replace, concat_ws, abs 
from pyspark.sql.functions import monotonically_increasing_id, upper, concat
from pyspark.sql.window import Window
from pyspark.sql import functions as f
import os 
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /usr/share/java/mariadb-java-client-2.5.3.jar pyspark-shell'

## 2. Crear una sesión de PySpark 

In [2]:
#Configuración de la sesión
spark_context = SparkContext()
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession

## 3. Exploración

Crear Dataframe para aeropuertos y vuelos

In [3]:
PATH = "./data/"

### 3.1. Ajuste información del archivo Aeropuertos.csv

In [4]:
df_aeropuertos = spark.read.load(PATH+"aeropuertosEtapa3.csv",format="csv", sep=",", inferSchema="true", header="true")
print(df_aeropuertos.schema)

StructType(List(StructField(sigla,StringType,true),StructField(iata,StringType,true),StructField(nombre,StringType,true),StructField(municipio,StringType,true),StructField(departamento,StringType,true),StructField(categoria,StringType,true),StructField(latitud,DoubleType,true),StructField(longitud,DoubleType,true),StructField(propietario,StringType,true),StructField(explotador,StringType,true),StructField(longitud_pista,DoubleType,true),StructField(ancho_pista,DoubleType,true),StructField(pbmo,DoubleType,true),StructField(elevacion,DoubleType,true),StructField(resolucion,StringType,true),StructField(fecha_construccion,StringType,true),StructField(fecha_vigencia,StringType,true),StructField(clase,StringType,true),StructField(tipo,StringType,true),StructField(numero_vuelos_origen,DoubleType,true),StructField(gcd_departamento,IntegerType,true),StructField(gcd_municipio,IntegerType,true),StructField(Ano,IntegerType,true)))


In [5]:
df_aeropuertos = df_aeropuertos.withColumn("longitud_pista",df_aeropuertos["longitud_pista"].cast(FloatType()))
df_aeropuertos = df_aeropuertos.withColumn("ancho_pista",df_aeropuertos["ancho_pista"].cast(FloatType()))
df_aeropuertos = df_aeropuertos.withColumn("pbmo",df_aeropuertos["pbmo"].cast(FloatType()))
df_aeropuertos = df_aeropuertos.withColumn("numero_vuelos_origen",df_aeropuertos["numero_vuelos_origen"].cast('int'))
df_aeropuertos = df_aeropuertos.withColumn("gcd_departamento",df_aeropuertos["gcd_departamento"].cast(StringType()))
df_aeropuertos = df_aeropuertos.withColumn("gcd_municipio",df_aeropuertos["gcd_municipio"].cast(StringType()))
df_aeropuertos = df_aeropuertos.withColumn('fecha_construccion',to_date(df_aeropuertos.fecha_construccion, 'yyyy-mm-dd'))
df_aeropuertos = df_aeropuertos.withColumn('fecha_vigencia',to_date(df_aeropuertos.fecha_vigencia, 'yyyy-mm-dd'))
df_aeropuertos = df_aeropuertos.withColumn("resolucion",df_aeropuertos["resolucion"].cast(StringType()))
df_aeropuertos = df_aeropuertos.withColumn("latitud",df_aeropuertos["latitud"].cast(FloatType()))
df_aeropuertos = df_aeropuertos.withColumn("longitud",df_aeropuertos["longitud"].cast(FloatType()))
df_aeropuertos = df_aeropuertos.withColumn("Ano",df_aeropuertos["Ano"].cast(StringType()))

#Eliminar columnas y otros ajustes
df_aeropuertos = df_aeropuertos.drop(col('iata'))
df_aeropuertos = df_aeropuertos.fillna({'pbmo': 0, 'resolucion': 0, 'propietario': 0})
df_aeropuertos = df_aeropuertos.withColumn('id_aeropuerto', monotonically_increasing_id())

#Organizar columnas
df_aeropuertos = df_aeropuertos.select('id_aeropuerto','sigla','nombre','categoria','clase', 'tipo','municipio',
                                       'departamento','propietario', 'explotador','resolucion','fecha_construccion',
                                       'fecha_vigencia','Ano','latitud','longitud','elevacion','numero_vuelos_origen',
                                       'longitud_pista','ancho_pista','pbmo')

In [6]:
# IDENTIFICAR REGISTRO CON ESTADO ACTUAL DEL AEROPUERTO
#Se utiliza un Window para extraer, del dataframe pasado por parámetro #Es decir, por cada llave natural, extrae la última versión.
df2_aeropuertos = df_aeropuertos.alias('df2_aeropuertos')
window = Window.partitionBy(df2_aeropuertos['sigla']).orderBy(df2_aeropuertos['Ano'].desc())
df2_aeropuertos = df2_aeropuertos.select('*', f.rank().over(window).alias('rank')).filter(f.col('rank') == 1) 
df2_aeropuertos = df2_aeropuertos.drop('rank')
#Ajustar nombres en nuevo DF para poder hacer el join con el df original
df2_aeropuertos = df2_aeropuertos.selectExpr('id_aeropuerto','Ano as Ano_2')
#Realiza left join entre df_aeropuertos y df2_aeropuertos
df = df_aeropuertos.join(df2_aeropuertos, how = 'left', on ='id_aeropuerto')

#estado_actual = No, registros anteriores de modificación del aerorpuerto
#estado_actual = Si, último registro de modificación del aeropuerto
df = df.withColumn('estado_actual', f.when(df['Ano_2'].isNull(), 'No').otherwise('Si'))
df = df.withColumn('fecha_vigencia_final', f.when(df['estado_actual'] == 'Si', f.to_date(f.lit('2199-12-31'), 'yyyy-MM-dd')).
                   otherwise(f.to_date(concat(col('Ano'),f.lit('-12-31')), 'yyyy-MM-dd')))

# IDENTIFICAR PRIMER REGISTRO PARA ASIGNAR FECHA VIGENCIA INICIAL
#Se utiliza un Window para extraer, del dataframe pasado por parámetro #Es decir, por cada llave natural, extrae la última versión.
df2_aeropuertos = df_aeropuertos.alias('df2_aeropuertos')
window = Window.partitionBy(df2_aeropuertos['sigla']).orderBy(df2_aeropuertos['Ano'].asc())
df2_aeropuertos = df2_aeropuertos.select('*', f.rank().over(window).alias('rank')).filter(f.col('rank') == 1) 
df2_aeropuertos = df2_aeropuertos.drop('rank')
#Ajustar nombres en nuevo DF para poder hacer el join con el df original
df2_aeropuertos = df2_aeropuertos.selectExpr('id_aeropuerto','fecha_construccion as fecha_vigencia_inicial','Ano as Ano_3')
#Realiza left join entre df_aeropuertos y df2_aeropuertos
df = df.join(df2_aeropuertos, how = 'left', on ='id_aeropuerto')

#estado_inicial = No, registros posteriores de modificación del aerorpuerto
#estado_inicial = Si, primer registro de construcción del aeropuerto
df = df.withColumn('estado_inicial', f.when(df['Ano_3'].isNull(), 'No').otherwise('Si'))
df = df.withColumn('fecha_vigencia_inicial', f.when(df['estado_inicial'] == 'Si', df['fecha_construccion']).
                   otherwise(f.to_date(concat(col('Ano'),f.lit('-01-01')), 'yyyy-MM-dd')))

# Seleccionar columnas para archivo csv
df_aeropuertos = df.select('id_aeropuerto','sigla','nombre','categoria','clase','tipo','municipio','departamento',
 'propietario','explotador','resolucion','fecha_construccion','fecha_vigencia','Ano',
          'latitud','longitud','elevacion','numero_vuelos_origen','longitud_pista','ancho_pista','pbmo',
          'estado_actual','fecha_vigencia_inicial','fecha_vigencia_final')


In [7]:
df_aeropuertos.select('nombre','fecha_construccion','longitud_pista','Ano','estado_actual','fecha_vigencia_inicial','fecha_vigencia_final').show(20)

+--------------------+------------------+--------------+----+-------------+----------------------+--------------------+
|              nombre|fecha_construccion|longitud_pista| Ano|estado_actual|fecha_vigencia_inicial|fecha_vigencia_final|
+--------------------+------------------+--------------+----+-------------+----------------------+--------------------+
|             LA ISLA|        2015-01-05|        1079.0|2015|           No|            2015-01-05|          2015-12-31|
|             LA ISLA|        2015-01-05|        1321.0|2016|           No|            2016-01-01|          2016-12-31|
|        LA ESCONDIDA|        2013-01-26|        2141.0|2014|           No|            2013-01-26|          2014-12-31|
|        LA ESCONDIDA|        2013-01-26|        3853.0|2015|           No|            2015-01-01|          2015-12-31|
|        LA ESCONDIDA|        2013-01-26|        6667.0|2016|           No|            2016-01-01|          2016-12-31|
|           MORICHITO|        2009-01-07

In [8]:
# Creación tabla pivote para id_aeropuerto y sigla
df_aeropuertos_id = df.where(df['estado_actual'] == 'Si')
df_aeropuertos_id = df_aeropuertos_id.select('sigla','id_aeropuerto')

### 3.2. Ajuste información del archivo Matriz_distancias.csv

In [9]:
# df_distancias = spark.read.load(PATH+"matriz_distancias.csv",format="csv", sep=",", inferSchema="true", header="true")
# df_cobertura = spark.read.load(PATH+"cobertura.csv",format="csv", sep=",", inferSchema="true", header="true")

In [10]:
#print(df_distancias.schema)

In [11]:
#Número de columnas
# len(df_distancias.columns)
#Ajustar encabezados
# df_distancias = df_distancias.withColumnRenamed('Unnamed: 0', 'centro_poblado')
# df_distancias = df_distancias.drop('LA PONDEROSA', 'SAN LUIS', 'EL RODEO', 'CAMPO ALEGRE', 'AGUA BLANCA', 'EL CORAJE', 'BECERRIL')

In [12]:
# df_distancias = df_distancias.withColumn("LA ESCONDIDA",df_distancias["LA ESCONDIDA"].cast(FloatType()))

In [13]:
# centros poblados y departamentos de Matriz_distancias.csv
# df_centros_poblados = df_distancias.select('centro_poblado').distinct()

# centros poblados y departamentos de Aeropuertos.csv
# df_temporal = df_aeropuertos.select('centro_poblado').distinct()

# creación tabla Centros Poblados
# df_centros_poblados = df_centros_poblados.join(df_temporal, how = 'left', on = 'centro_poblado')

In [14]:
# df_centros_poblados.schema.names

In [15]:
# df_aeropuertos.schema.names

### 3.3. Ajuste información del archivo Vuelos.csv

In [16]:
df_vuelos = spark.read.load(PATH+"vuelosEtapa3.csv",format="csv", sep=",", inferSchema="true", header="true")
print(df_vuelos.schema)

StructType(List(StructField(ano,IntegerType,true),StructField(mes,IntegerType,true),StructField(origen,StringType,true),StructField(destino,StringType,true),StructField(tipo_equipo,StringType,true),StructField(tipo_vuelo,StringType,true),StructField(trafico,StringType,true),StructField(empresa,StringType,true),StructField(vuelos,IntegerType,true),StructField(sillas,IntegerType,true),StructField(carga_ofrecida,DoubleType,true),StructField(pasajeros,IntegerType,true),StructField(carga_bordo,DoubleType,true)))


In [17]:
df_vuelos.select([count(when(isnan(c), c)).alias(c) for c in df_vuelos.columns]).show()

+---+---+------+-------+-----------+----------+-------+-------+------+------+--------------+---------+-----------+
|ano|mes|origen|destino|tipo_equipo|tipo_vuelo|trafico|empresa|vuelos|sillas|carga_ofrecida|pasajeros|carga_bordo|
+---+---+------+-------+-----------+----------+-------+-------+------+------+--------------+---------+-----------+
|  0|  0|     0|      0|          0|         0|      0|      0|     0|     0|             0|        0|          0|
+---+---+------+-------+-----------+----------+-------+-------+------+------+--------------+---------+-----------+



In [18]:
df_vuelos = df_vuelos.withColumn("vuelos",df_vuelos["vuelos"].cast('int'))
df_vuelos = df_vuelos.withColumn("sillas",df_vuelos["sillas"].cast('int'))
df_vuelos = df_vuelos.withColumn("pasajeros",df_vuelos["pasajeros"].cast('int'))
df_vuelos = df_vuelos.withColumn("carga_ofrecida",df_vuelos["carga_ofrecida"].cast(FloatType()))
df_vuelos = df_vuelos.withColumn("carga_bordo",df_vuelos["carga_bordo"].cast(FloatType()))
df_vuelos = df_vuelos.withColumn("ano",df_vuelos["ano"].cast(StringType()))
df_vuelos = df_vuelos.withColumn("mes",df_vuelos["mes"].cast(StringType()))

In [19]:
df_vuelos = df_vuelos.fillna({'sillas': 0, 'pasajeros': 0})
df_vuelos = df_vuelos.withColumn('trafico', regexp_replace('trafico', 'nan', 'N'))

In [20]:
df_vuelos.select([count(when(isnan(c), c)).alias(c) for c in df_vuelos.columns]).show()

+---+---+------+-------+-----------+----------+-------+-------+------+------+--------------+---------+-----------+
|ano|mes|origen|destino|tipo_equipo|tipo_vuelo|trafico|empresa|vuelos|sillas|carga_ofrecida|pasajeros|carga_bordo|
+---+---+------+-------+-----------+----------+-------+-------+------+------+--------------+---------+-----------+
|  0|  0|     0|      0|          0|         0|      0|      0|     0|     0|             0|        0|          0|
+---+---+------+-------+-----------+----------+-------+-------+------+------+--------------+---------+-----------+



In [21]:
ajuste_mes = {'1':'01', '2':'02', '3':'03', '4':'04', '5':'05', '6':'06', '7':'07', '8':'08', '9':'09'}
df_vuelos = df_vuelos.na.replace(ajuste_mes,2,'mes')

In [22]:
df_vuelos = df_vuelos.withColumn("ano_mes", concat_ws("-",'ano','mes'))

In [23]:
# Ajustar valores de sillas o carga desaprovechada, no pueden ser negativos

df_vuelos = df_vuelos.withColumn(
    'sillas_desaprovechadas',
    when(((col("sillas")) - (col("pasajeros")) < 0), 0)
    .otherwise((col("sillas")) - (col("pasajeros"))))

df_vuelos = df_vuelos.withColumn(
    'carga_desaprovechada',
    when(((col("carga_ofrecida")) - (col("carga_bordo")) < 0), 0)
    .otherwise((col("carga_ofrecida")) - (col("carga_bordo"))))

In [24]:
df_vuelos = df_vuelos.withColumn('id_vuelo', monotonically_increasing_id())

In [25]:
# Crear tablas y organizar columnas

df_atributos_vuelo = df_vuelos.select('id_vuelo','origen','destino','tipo_equipo','empresa','tipo_vuelo','trafico')

df_fecha = df_vuelos.select('ano_mes','ano','mes')
df_fecha = df_fecha.dropDuplicates((['ano_mes'])).sort('ano_mes')

df_vuelos = df_vuelos.select('id_vuelo','ano_mes','vuelos','pasajeros','sillas','sillas_desaprovechadas',
                             'carga_ofrecida','carga_bordo','carga_desaprovechada','tipo_equipo')

In [26]:
df_atributos_vuelo.schema.names

['id_vuelo',
 'origen',
 'destino',
 'tipo_equipo',
 'empresa',
 'tipo_vuelo',
 'trafico']

In [27]:
df_fecha.schema.names

['ano_mes', 'ano', 'mes']

In [28]:
df_vuelos.schema.names

['id_vuelo',
 'ano_mes',
 'vuelos',
 'pasajeros',
 'sillas',
 'sillas_desaprovechadas',
 'carga_ofrecida',
 'carga_bordo',
 'carga_desaprovechada',
 'tipo_equipo']

### 3.5 Ajuste información del archivo Tipo de equipo

In [29]:
df_tipo_equipo_fuente = spark.read.load(PATH+"doc8643AircraftTypes.csv",format="csv", sep=",", inferSchema="true", header="true")
print(df_tipo_equipo_fuente.schema)

StructType(List(StructField(AircraftDescription,StringType,true),StructField(Description,StringType,true),StructField(Designator,StringType,true),StructField(EngineCount,StringType,true),StructField(EngineType,StringType,true),StructField(ManufacturerCode,StringType,true),StructField(ModelFullName,StringType,true),StructField(WTC,StringType,true)))


In [30]:
df_tipo_equipo_fuente.show(5)

+-------------------+-----------+----------+-----------+----------+--------------------+--------------+---+
|AircraftDescription|Description|Designator|EngineCount|EngineType|    ManufacturerCode| ModelFullName|WTC|
+-------------------+-----------+----------+-----------+----------+--------------------+--------------+---+
|          LandPlane|        L2J|      J328|          2|       Jet|328 SUPPORT SERVICES|Dornier 328JET|  M|
|          LandPlane|        L1P|      UL45|          1|    Piston|              3XTRIM|     450 Ultra|  L|
|          LandPlane|        L1P|      UL45|          1|    Piston|              3XTRIM|         Ultra|  L|
|          LandPlane|        L1P|      TR55|          1|    Piston|              3XTRIM|    550 Trener|  L|
|          LandPlane|        L1P|      TR55|          1|    Piston|              3XTRIM|      Trener  |  L|
+-------------------+-----------+----------+-----------+----------+--------------------+--------------+---+
only showing top 5 rows



In [31]:
# Verificación de datos nulos
df_tipo_equipo_fuente.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_tipo_equipo_fuente.columns]).show()

+-------------------+-----------+----------+-----------+----------+----------------+-------------+---+
|AircraftDescription|Description|Designator|EngineCount|EngineType|ManufacturerCode|ModelFullName|WTC|
+-------------------+-----------+----------+-----------+----------+----------------+-------------+---+
|                  0|          0|         0|          0|         0|               0|            0|  0|
+-------------------+-----------+----------+-----------+----------+----------------+-------------+---+



In [32]:
# Perfilamiento AircraftDescription

df_tipo_equipo_fuente.select('AircraftDescription').distinct().show(5)
print(df_tipo_equipo_fuente.select('AircraftDescription').distinct().count())

+-------------------+
|AircraftDescription|
+-------------------+
|          LandPlane|
|           SeaPlane|
|         Gyrocopter|
|         Helicopter|
|          Tilt-wing|
+-------------------+
only showing top 5 rows

6


In [33]:
# Perfilamiento Description

df_tipo_equipo_fuente.select('Description').distinct().show(5)
df_tipo_equipo_fuente.select('Description').distinct().count()

+-----------+
|Description|
+-----------+
|        A1P|
|        L2T|
|        L2E|
|        L4P|
|        L3J|
+-----------+
only showing top 5 rows



37

In [34]:
# Perfilamiento Designator

df_tipo_equipo_fuente.select('Designator').distinct().show(5)
df_tipo_equipo_fuente.select('Designator').distinct().count()

+----------+
|Designator|
+----------+
|      AB18|
|      VO10|
|      LAKX|
|       A23|
|      BE30|
+----------+
only showing top 5 rows



2640

In [35]:
# Perfilamiento EngineCount

df_tipo_equipo_fuente.select('EngineCount').distinct().show(5)
df_tipo_equipo_fuente.select('EngineCount').distinct().count()

+-----------+
|EngineCount|
+-----------+
|          3|
|          8|
|          6|
|          C|
|          1|
+-----------+
only showing top 5 rows



7

In [36]:
# Perfilamiento EngineType

df_tipo_equipo_fuente.select('EngineType').distinct().show(5)
df_tipo_equipo_fuente.select('EngineType').distinct().count()

+--------------------+
|          EngineType|
+--------------------+
|Turboprop/Turboshaft|
|              Rocket|
|                 Jet|
|            Electric|
|              Piston|
+--------------------+



5

In [37]:
# Perfilamiento ManufacturerCode

df_tipo_equipo_fuente.select('ManufacturerCode').distinct().show(5)
df_tipo_equipo_fuente.select('ManufacturerCode').distinct().count()

+----------------+
|ManufacturerCode|
+----------------+
|           GREGA|
|      HOWARD (2)|
|         JOHNSON|
|         OMA SUD|
|  TEXAS AIRPLANE|
+----------------+
only showing top 5 rows



1539

In [38]:
# Perfilamiento ModelFullName

df_tipo_equipo_fuente.select('ModelFullName').distinct().show(5)
df_tipo_equipo_fuente.select('ModelFullName').distinct().count()

+-------------+
|ModelFullName|
+-------------+
|      Loisirs|
|   Loadmaster|
|        Z-131|
|     15 Sedan|
|        H-175|
+-------------+
only showing top 5 rows



7660

In [39]:
# Perfilamiento WTC

df_tipo_equipo_fuente.select('WTC').distinct().show(5)
df_tipo_equipo_fuente.select('WTC').distinct().count()

+---+
|WTC|
+---+
|  M|
|  L|
|L/M|
|  H|
+---+



4

In [40]:
df_tipo_equipo_fuente = df_tipo_equipo_fuente.selectExpr('Designator as tipo_equipo','AircraftDescription as tipo_aeronave',
                                                         'Description as codigo_descripcion', 'EngineCount',
                                                         'EngineType as numero_motores', 'ManufacturerCode as fabricante',
                                                         'WTC')
df_tipo_equipo_fuente = df_tipo_equipo_fuente.distinct()

#Se utiliza un Window para extraer, del dataframe pasado por parámetro #Es decir, por cada llave natural, extrae la última versión.
df2_tipo_equipo_fuente = df_tipo_equipo_fuente.alias('df2_tipo_equipo_fuente')
window = Window.partitionBy(df2_tipo_equipo_fuente['tipo_equipo']).orderBy(df2_tipo_equipo_fuente['fabricante'].desc())
df2_tipo_equipo_fuente = df2_tipo_equipo_fuente.select('*', f.rank().over(window).alias('rank')).filter(f.col('rank') == 1) 
df2_tipo_equipo_fuente = df2_tipo_equipo_fuente.drop('rank')
print((df2_tipo_equipo_fuente.count(), len(df2_tipo_equipo_fuente.columns)))
df2_tipo_equipo_fuente.schema.names

(2640, 7)


['tipo_equipo',
 'tipo_aeronave',
 'codigo_descripcion',
 'EngineCount',
 'numero_motores',
 'fabricante',
 'WTC']

In [41]:
# Perfilamiento Tipo de equipo de tabla vuelos
df_tipo_equipo_vuelos = df_vuelos.select('tipo_equipo').distinct()
df_tipo_equipo_vuelos.select('tipo_equipo').count()

140

In [42]:
#Ajuste tabla con dimensión Tipo de equipo

df_tipo_equipo = df2_tipo_equipo_fuente.join(df_tipo_equipo_vuelos, how = 'rightouter', on ='tipo_equipo')
df_tipo_equipo = df_tipo_equipo.na.fill("-")
df_tipo_equipo.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_tipo_equipo.columns]).show()
print((df_tipo_equipo.count(), len(df_tipo_equipo.columns)))
df_tipo_equipo.schema.names

+-----------+-------------+------------------+-----------+--------------+----------+---+
|tipo_equipo|tipo_aeronave|codigo_descripcion|EngineCount|numero_motores|fabricante|WTC|
+-----------+-------------+------------------+-----------+--------------+----------+---+
|          0|            0|                 0|          0|             0|         0|  0|
+-----------+-------------+------------------+-----------+--------------+----------+---+

(140, 7)


['tipo_equipo',
 'tipo_aeronave',
 'codigo_descripcion',
 'EngineCount',
 'numero_motores',
 'fabricante',
 'WTC']

In [43]:
df_tipo_equipo.select('tipo_equipo').count()

140

### 3.6. Ajuste información del archivo Cobertura.csv

In [44]:
# print(df_cobertura.schema)

In [45]:
#Número de columnas
# len(df_cobertura.columns)
#Ajustar encabezados
# df_cobertura = df_cobertura.withColumnRenamed('Aeropuerto', 'aeropuerto')
# df_cobertura = df_cobertura.withColumnRenamed('Distancia(Km)','Distancia')
# df_cobertura.schema.names

In [46]:
# df_cobertura = df_cobertura.withColumn("Distancia",df_cobertura["Distancia"].cast(FloatType()))
# df_cobertura = df_cobertura.withColumn("D_Aerodromo",df_cobertura["D_Aerodromo"].cast(FloatType()))
# df_cobertura = df_cobertura.withColumn("D_Regional",df_cobertura["D_Regional"].cast(FloatType()))
# df_cobertura = df_cobertura.withColumn("D_Nacional",df_cobertura["D_Nacional"].cast(FloatType()))
# df_cobertura = df_cobertura.withColumn("D_Internacional",df_cobertura["D_Internacional"].cast(FloatType()))

### 3.7. Creación archivos csv

In [47]:
#df_aeropuertos.toPandas().to_csv(PATH+'Aeropuertos_ETL_Historia.csv')
#df_vuelos.toPandas().to_csv(PATH+'Vuelos_ETL_Historia.csv')
#df_atributos_vuelo.toPandas().to_csv(PATH+'Atributos_vuelo_ETL_HistoriaI.csv')
#df_fecha.toPandas().to_csv(PATH+'Fecha_ETL_Historia.csv')
#df_aeropuertos_id.toPandas().to_csv(PATH+'Aeropuertos_ID_ETL_Historia.csv')
df_tipo_equipo.toPandas().to_csv(PATH+'Tipo_equipo.csv')

# Se pidieron en la semana 4 pero los descartaron en la Semana 5
#df_distancias.toPandas().to_csv(PATH+'Matriz_distancias_ETL_Historia.csv')
#df_cobertura.toPandas().to_csv(PATH+'Cobertura_ETL_Historia.csv')
#df_centros_poblados.toPandas().to_csv(PATH+'Centros_Poblados_ETL_Historia.csv')

### 3.7. Cálculos

In [48]:
#df_vuelos.groupBy('ano').agg({'ano': 'count'}).sort('ano').show()

In [49]:
#df_vuelos.filter((df_vuelos['pasajeros'] >= 0) & (df_vuelos['sillas'] >= 0)).approxQuantile(["pasajeros", "sillas", "carga_ofrecida", "carga_bordo"], [0.5], 0.25)

In [50]:
#df_aeropuertos.select('fecha_construccion').show()

In [51]:
#df_aeropuertos.groupBy('fecha_construccion').agg({'fecha_construccion': 'count'}).show()

In [52]:
#df_aeropuertos.describe('ancho_pista','pbmo').show()

In [53]:
#df_aeropuertos.select('pbmo').filter(df_aeropuertos['pbmo'] >= 0).show()

In [54]:
spark.stop()